In [2]:
from Classes import * 
from Simu import *
from NLL import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
from scipy import integrate
import sympy as sy
from scipy import linalg
import pylab as pl
import math as ma
import time 
%matplotlib inline

# ---------- Fixés pour l'expérience -------------------
# Nombre de signaux longitudinaux
D=1
# Paramètres de variance pour la génération de paramètres du signal longitudinal
varu=1.
# Ecart-Type de l'erreur gaussienne des observations de Y
sigY=0.
# Numéro associé à la fonction paramètre génératrice
true_beta=beta(num=1)
numbeta=1
# Nombre de variables scalaires
p=1
# Options d'optimisation du critère d'inférence
opts0={'disp': True, 'maxiter': 120, 'gtol':200.}
# Taille de la grille d'évaluation de l'AMSE
nb=30
# Taille de la grille de tracé du graphe
sizegrid=50
# ---------- Variables de l'expérience -------------------
vecN=[130,260,650]
vecn=[10,100]
vecgam=[0.,1.]
vecKb=[5,8]
vecJ=[100]
# Finir exp avec N 200,500, pour n =100, J=70 et rep=10
rep=40
rand_init=1
# Sauvegarde
save=False

vecRep=range(rep)
vecInit=range(rand_init)

candidats=[0,1,3,5,6]
N_modeles=10


Simu=expandnp([vecRep,vecN,vecn,vecgam])
Simu=Simu.astype(int)
N_Simu=len(Simu)
print("Design Simus")
print(" Rep   N  n  Erreur  n°Simu")
print(np.hstack((np.asarray(Simu),np.arange(N_Simu).reshape((N_Simu,1)))))

Exp=expandnp([vecInit,vecJ,vecKb,range(N_Simu)])
Exp=Exp.astype(int)
print("Design Params")
print("Init J Kbeta n°Simu")
print(np.asarray(Exp))
N_exp=len(Exp)
print(" Nombre d'inférences à réaliser : "+str(N_exp))
b_liste=[[] for i in range(N_exp) ]
messages=[[] for i in range(N_exp) ]
Jac=[[] for i in range(N_exp) ]
V_It_d_AMSE=np.zeros((6,N_modeles,N_exp),float)


/home/diallo/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Design Simus
 Rep   N  n  Erreur  n°Simu
[[  0 130  10   0   0]
 [  1 130  10   0   1]
 [  2 130  10   0   2]
 ..., 
 [ 37 650 100   1 477]
 [ 38 650 100   1 478]
 [ 39 650 100   1 479]]
Design Params
Init J Kbeta n°Simu
[[  0 100   5   0]
 [  0 100   8   0]
 [  0 100   5   1]
 ..., 
 [  0 100   8 478]
 [  0 100   5 479]
 [  0 100   8 479]]
 Nombre d'inférences à réaliser : 960


In [3]:
# Intro pandas
Longi=pd.read_csv("Longi.csv",header=0,names=['t','val','j','idv','simu'])
Data=pd.read_csv("Data.csv",header=0,names=['tmax','y','N','n','Err','rep','simu'])
print(Data.head())
#print(Longi.head())
#print(Data.iloc[[0,2],[2,3]])
#print(Data[(Data["simu"]==0)&(Data["tmax"]<0.45)])

       tmax         y      N     n  Err  rep  simu
0  0.733473  0.561200  130.0  10.0  0.0  0.0   0.0
1  0.619155 -1.067111  130.0  10.0  0.0  0.0   0.0
2  0.646300  1.287698  130.0  10.0  0.0  0.0   0.0
3  0.179247  0.483030  130.0  10.0  0.0  0.0   0.0
4  0.391991  1.041657  130.0  10.0  0.0  0.0   0.0


In [ ]:
nbsim=Data["simu"].unique().size

Params=expandnp([vecKb,vecJ])
numParam=len(Params[:,0])
deb=time.clock()
l_L=[ [] for sim in range(nbsim)]
l_Xdata=[ [] for sim in range(nbsim)]
l_T=[ [] for sim in range(nbsim)]
l_Y=[ [] for sim in range(nbsim)]
for simu in range(nbsim):
    #print("simu n°"+str(simu)+" sur "+str(nbsim-1)+" duree actuelle :"+str(time.clock()-deb))
    N=int(Data["N"][Data["simu"]==simu].iloc[0])
    l_Y[simu]=np.asarray( Data["y"][Data["simu"]==simu] )
    l_T[simu]=np.asarray( Data["tmax"][Data["simu"]==simu] )
    l_L[simu]=[ [[] for j in range(D) ]  for i in range(N) ]
    l_Xdata[simu]=[ [[] for j in range(D) ]  for i in range(N) ]
    Xda=Longi[Longi["simu"]==simu]
    for i in range(N):
        l_L[simu][i][0]=np.asarray( Xda["t"][Xda["idv"]==i] )
        l_Xdata[simu][i][0]=np.asarray( Xda["val"][Xda["idv"]==i] )

In [ ]:
ref_sim=-1
ref_Kb=-1
ref_J=-1
k=0
interrupteur=1
Res=pd.DataFrame(np.zeros((1,11),float), columns=['Init','J','Kbeta','sim','model','vrais','n_it','dur','emg','ise','sce'])
for Init,J,Kbeta,sim in Exp:
    k+=1
    if sim!=ref_sim:
        ref_sim=sim
        Rep=Simu[sim,0]
        N=Simu[sim,1]
        lim_learn=int(round(0.77*N))
        print("N learn ="+str(lim_learn))
        n_t_moy=Simu[sim,2]
        Err=Simu[sim,3]
        print("---------------------------- Simu n°"+str(sim)+" -> N ="+str(N)+"; n_moy ="+str(n_t_moy)+"Erreur ="+str(Err)+" Repetition ="+str(Rep))
        (T_full,L_full,Xdata_full,Y_full)=(l_T[sim],l_L[sim],l_Xdata[sim],l_Y[sim])
        ( T , L , Xdata , Y ) = (T_full[0:lim_learn],L_full[0:lim_learn],Xdata_full[0:lim_learn],Y_full[0:lim_learn])
        Y_test= Y_full[lim_learn:N]
        interrupteur=1
        Res.to_csv("Res.csv")
    print("Exp n°"+str(k)+" sur "+str(N_exp) +"-> N : "+str(N)+"; n_t_moy : "+str(n_t_moy)+"; Kbeta : "+str(Kbeta)+"; J : "+str(J))
    if Kbeta!=ref_Kb:
        ref_Kb=Kbeta
        print("---------------------------- Kbeta ="+str(Kbeta))
        Beta_fonc_est=construct_beta(Kbeta**2)
        interrupteur=1
    if J!=ref_J:
        ref_J=J
        print("---------------------------- J ="+str(J))
        interrupteur=1
    if interrupteur==1:
        (l_full,V_full,I_pen)=Pre_Comp(L_full,T_full,Xdata_full,Y_full,Kbeta,J)
        erreur=0
        ( Phi_mat,k_fonc_yx,k_der,I_pen_yx )=Pre_Comp_YX(L,T,Xdata,Y,Kbeta,J)
        (l,l_test) = ( l_full[0:lim_learn,:] , l_full[lim_learn:N,:] )
        ( V , V_test ) =( V_full[0:lim_learn] , V_full[lim_learn:N] )
        n_par=len(k_der)
        interrupteur=0
    print("Initialisation n°"+str(Init))
    b=np.random.uniform(-10,10,D*Kbeta**2)
    (Z,Z_test)=( np.zeros((lim_learn,p),float) , np.zeros((N-lim_learn,p),float) ) 
    print("std Y ="+str(np.std(Y)))
    for mod in candidats:
        emg=0.
        n_it=0.
        try:
            if mod==0:
                # --------------------------MODELE 0 Estimateur MC Classique ------------
                deb=time.clock()
                bf=MCO_best(l,Y)
                Pf=np.concatenate((np.zeros(p,float),bf))
                dur=time.clock()-deb 
                rmse=rMSE( Pred_YlX(Pf,Z_test,l_test) , Y_test )
            if mod==1:
                # --------------------------MODELE 1 Estimateur MC pénalisé ------------
                deb=time.clock()
                a=MCO_pen_learn(Z,l,Y,I_pen,n_test=10)
                Lambda=a[2]
                bf=MCO_pen_best(l,Y,I_pen,Lambda)
                Pf=np.concatenate((np.zeros(p,float),bf))
                dur=time.clock()-deb 
                rmse=rMSE( Pred_YlX(Pf,Z_test,l_test) , Y_test )
            if mod==2:
                # --------------------------MODELE 2 Estimateur MC+pénalitévariance ------------
                deb=time.clock()
                bf=MCO_penvar_best(V,l,Y)
                Pf=np.concatenate((np.zeros(p,float),bf))
                dur=time.clock()-deb 
                rmse=rMSE( Pred_YlX(Pf,Z_test,l_test) , Y_test )
            if mod==3:
                # --------------------------MODELE 3 Estimateur MC+double pénalité ------------
                deb=time.clock()
                a=MCO_2pen_learn(Z,V,l,Y,I_pen,n_test=10)
                Lambda=a[2]
                bf=MCO_2pen_best(Z,V,l,Y,I_pen,Lambda)
                Pf=np.concatenate((np.zeros(p,float),bf))
                dur=time.clock()-deb 
                rmse=rMSE( Pred_YlX(Pf,Z_test,l_test) , Y_test )
            if mod==4:
                # --------------------------MODELE 4 L_YlX pénalisé ------------
                Lambda=N*1./(Kbeta**2)
                deb=time.clock()
                (Pf,Vf,Gf,vrais)=LYlX_optim(L_YlX_pen,Z,l,V,Y,I_pen,Lambda,n_it=200,n_ini=10,tol=0.5)
                dur=time.clock()-deb 
                rmse=rMSE( Pred_YlX(Pf,Z_test,l_test) , Y_test )
                emg=np.sqrt( Gf.dot(Gf)/len(Gf))
            if mod==5:
                # --------------------------MODELE 5 L_YlX pénalisé stochastique ---------------------
                deb=time.clock()
                a=LYlX_learn(Z,V,l,Y,I_pen,n_it=N*50,n_ini=4,tol=0.2,n_test=10)
                Lambda=a[2]
                (Pf,Vf,Gf,vrais)=LYlX_optim(S_L_YlX_pen,Z,l,V,Y,I_pen,Lambda,n_it=100*N,n_ini=5,tol=0.2)
                dur=time.clock()-deb 
                rmse=rMSE( Pred_YlX(Pf,Z_test,l_test) , Y_test )
                emg=np.sqrt( Gf.dot(Gf)/len(Gf))
            if mod==6:
                #-------------------------- MODELE 6 LYX stochastique pénalisé
                deb=time.clock()
                a=LYX_learn(Z,l,L,T,Xdata,Y,Phi_mat,k_fonc_yx,k_der,I_pen,n_it=N*5,n_ini=2,tol=0.2,n_test=5)
                Lambda=a[2]
                (Pf,Vf,Gf,vrais)=LYX_optim(Z,l,L,T,Xdata,Y,Phi_mat,k_fonc_yx,k_der,I_pen,Lambda,
                                           n_it=30*N,n_ini=3,tol=0.2)
                dur=time.clock()-deb
                rmse=rMSE( Pred_YlX(Pf[0:(D*Kbeta**2+p)],Z_test,l_test) , Y_test )
                emg=np.sqrt( Gf.dot(Gf)/len(Gf))

            #--------------- STOCKAGE    
            Vrais_ref=L_YlX(Pf[0:(D*Kbeta**2+p)],Z,V,l,Y,D)[0]
            b_l=Pf[p:(p+Kbeta**2)]
            emb=np.sqrt(b_l.dot(b_l)/len(b_l))
            b_learned=b_l.reshape((D,Kbeta**2)).T
            ise=ISE(numbeta,Beta_fonc_est,b_learned,T,nb=30)
            if mod in [0,1,2,3]:
                Vf=Vrais_ref
            print(str(mod)+" Vf :"+str(Vf)+" rMSE :"+str(rmse)+" EMG :"+str(emg)+" duree :"+str(dur)+" ISE :"+str(ise)+" EMb :"+str(emb) )
            Res=Res.append({'Init':Init,'J':J,'Kbeta':Kbeta,'sim':sim,'model':mod,'vrais':Vf,'n_it':n_it,'dur':dur,'emg':emg,'ise':ise,'rmse':rmse,'N':N,'n':n_t_moy,'err':Err},ignore_index=True)
        except:
            #--------------- STOCKAGE    
            print(str(mod)+" ->Erreur dans l'inférence")
            Res=Res.append({'Init':Init,'J':J,'Kbeta':Kbeta,'sim':sim,'model':mod,'vrais':float('nan'),'n_it':float('nan'),'dur':float('nan'),'emg':float('nan'),'ise':float('nan'),'rmse':float('nan'),'N':N,'n':n_t_moy,'err':Err},ignore_index=True)
            pass
    print("")
    

N learn =100
---------------------------- Simu n°0 -> N =130; n_moy =10Erreur =0 Repetition =0
Exp n°1 sur 960-> N : 130; n_t_moy : 10; Kbeta : 5; J : 100
---------------------------- Kbeta =5
---------------------------- J =100
Initialisation n°0
std Y =0.929765459427
ISE : 33.9096548902
0 Vf :80.4369258934 rMSE :0.945839582265 EMG :0.0 duree :0.049236 ISE :33.9096548902 EMb :381528.284122
ISE : 9.27455227751
1 Vf :73.0806134091 rMSE :0.960525635915 EMG :0.0 duree :0.048813 ISE :9.27455227751 EMb :0.2792081487
ISE : 8.68291999323
2 Vf :66.1850933185 rMSE :0.968257194437 EMG :0.0 duree :0.075708 ISE :8.68291999323 EMb :147740.317688
ISE : 8.4854701335
3 Vf :75.2234168413 rMSE :0.976165285026 EMG :0.0 duree :0.643428 ISE :8.4854701335 EMb :0.0334490109516
ISE : 12.490606081
4 Vf :773.879933862 rMSE :1.35511600311 EMG :38.1529800618 duree :15.342773 ISE :12.490606081 EMb :3.27214050902
ISE : 9.27197836863
5 Vf :95.0875775811 rMSE :0.966439390063 EMG :25.7321836544 duree :21.263124 ISE :9